In [1]:
using Pkg
Pkg.activate("/home/ryansenne/PycharmProjects/ssm_julia")

  Activating project at `~/PycharmProjects/ssm_julia`


In [2]:
# include("/Users/ryansenne/Documents/Github/SSM//src/SSM.jl")
include("/home/ryansenne/PycharmProjects/ssm_julia/src/SSM.jl")
using Random
using .SSM
using Distributions

### Gaussian GLM Example

In [10]:
# Fit a Gaussian GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]
y = X_concat * β + rand(Normal(0, 1), 10000)

# define a GLM
glm = GaussianRegression(X, y)
fit!(glm, LSELoss(), 1000)

4-element Vector{Float64}:
  2.02647729897031
  3.0079791837279575
 -4.041013313373754
  4.991249502322808

### Poisson GLM Example

In [8]:
# Fit a Poisson GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Poisson.(exp.(X_concat * β))))

# define a GLM
glm = PoissonRegression(X, y)
fit!(glm, PoissonLoss(), 1000)


4-element Vector{Float64}:
  2.000992921834151
  3.0020310084238937
 -4.0003594268560185
  4.996621009202876

### Binomial GLM Example (i.e. logistic regression)

In [7]:
# sigmoid function
function sigmoid(x)
    return 1 / (1 + exp(-x))
end

# Fit a Binomial GLM
# Generate some synthetic data
X = rand(10000, 3); # 1000 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Binomial.(1, sigmoid.(X_concat * β))))

# define a GLM
glm = BinomialRegression(X, y)
fit!(glm, CrossEntropyLoss(), 1000)

4-element Vector{Float64}:
  1.9150124819278913
  3.0577957045910296
 -3.8336980061602524
  4.895904304852471

In [3]:
# try out markov glm

# Generate synthetic data from two distinct regimes
X = rand(10000, 3); # 1000 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β₁ = [2, 3, -4, 5]
β₂ = [2, 3, 4, 5]

# Generate some synthetic data
y₁ = X_concat * β₁ + rand(Normal(0, 1), 10000)
y₂ = X_concat * β₂ + rand(Normal(0, 1), 10000)

# concatenate X1 and X2
x = vcat(X_concat, X_concat)

# concatenate y1 and y2
y = vcat(y₁, y₂)

# define a Gaussian HMMGLM
glm = GaussianMarkovRegression(y, x, 2)

MethodError: MethodError: no method matching GaussianMarkovRegression(::Vector{Float64}, ::Matrix{Float64}, ::Int64, ::Vector{GaussianRegression{Float64}}, ::HMM{GaussianEmission})

Closest candidates are:
  GaussianMarkovRegression(::Vector{T}, ::Matrix{T}, ::Int64, !Matched::Vector{GaussianRegression}, ::HMM) where T<:Real
   @ Main.SSM ~/PycharmProjects/ssm_julia/src/MarkovRegression.jl:19
  GaussianMarkovRegression(::Vector{T}, ::Matrix{T}, ::Int64) where T<:Real
   @ Main.SSM ~/PycharmProjects/ssm_julia/src/MarkovRegression.jl:30
